In [38]:
from IPython.display import Markdown, display
from openai import OpenAI
import json
import os
from time import sleep
import multiprocessing
from multiprocessing.pool import ThreadPool

In [39]:
client = OpenAI()

In [40]:
gpt4o = "gpt-4o-2024-08-06"
gpt4o_mini = "gpt-4o-mini-2024-07-18"
small_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-small-dataset:AYDdDFb6"
medium_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-medium-dataset:AYE8fROA"
large_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-large-dataset:AYEAuS9Z"

small_simple_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:simple-small-dataset:AYWlgHow"
medium_simple_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:simple-medium-dataset:AYWlf8C2"
large_simple_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:simple-large-dataset:AYWqFlro"

large_2_epochs_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:large-dataset-2-epochs:AYWvOBRa"

In [59]:
model_used = large_2_epochs_dataset_model

In [60]:
# Create the system_prompt
example_simple = """User input: "חזון ישעיהו בןאמוץ אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"
Expected output: ["חזון ישעיהו בןאמוץ","אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"]"""

example_complex = """User input: "יביא יהוה עליך ועלעמך ועלבית אביך ימים אשר לאבאו למיום סוראפרים מעל יהודה את מלך אשור פ"
Expected output: ["יביא יהוה עליך ועלעמך ועלבית אביך ימים את מלך אשור פ" ,"אשר לאבאו למיום" ,"סוראפרים מעל יהודה"]"""


SYSTEM_PROMPT = f"Identify clauses in this Biblical Hebrew verse and return a JSON list containing the clauses, as shown in the following examples. Do not write anything else than the JSON list in your output. Example 1 {example_simple} Example 2 {example_complex}"

In [1]:
# Functions

def parse_verse(verse):
    """Send a request to the chosen OpenAI model to parse a Biblical Hebrew verse into clauses."""
    print("Parsing verse:", verse)
    for _ in range(10):
        try:
            response = client.chat.completions.create(
                model=model_used,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"{verse}"}
                ]
            )
            # Extract and return the response
            return verse, response.choices[0].message.content
        except Exception as e:
            print(f"Error processing verse: {verse}")
            print(e)
            sleep(5)
            print("Failed to call OpenAI. Retrying.")


def process_file(input_file, output_file):
    """Read a JSONL file containing verses, process them and save results to another JSONL file."""
    verses = []
    with open(input_file, "r", encoding="utf-8") as infile:
        for line in infile:
            data = json.loads(line)
            verses.append(data["messages"][1]["content"])
    pool = ThreadPool(processes=5)
    results = pool.map(parse_verse, verses)
    
    with open(output_file, "w", encoding="utf_8") as outfile:
        for verse, result in results:
            if result:
                output = {"verse": verse, "parsed_clauses": result}
                outfile.write(json.dumps(output, ensure_ascii=False) + "\n")

In [62]:
# Input and output files
input_jsonl_file = "data/fine_tuning_datasets/trial_3/validation_large.jsonl"
output_jsonl_file = "data/fine_tuning_datasets/trial_3/model_outputs/output_2_epochs_large_model.jsonl"

In [63]:
print(model_used)

ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:large-dataset-2-epochs:AYWvOBRa


In [ ]:
%%time
process_file(input_jsonl_file, output_jsonl_file)

Parsing verse:Parsing verse: Parse this verse: ויקח מידם ויצר אתו בחרט ויעשהו עגל מסכה ויאמרו אלה אלהיך ישראל אשר העלוך מארץ מצרים
Parsing verse: Parse this verse: וישאהו מבית אבינדב אשר בגבעה עם ארון האלהים ואחיו הלך לפני הארון
 Parse this verse: ויצוו אתבני בנימן לאמר לכו וארבתם בכרמים
Parsing verse: Parse this verse: בעת ההיא שלח המלך אחז עלמלכי אשור לעזר לו
Parsing verse: Parse this verse: בשנת שלשים ושש למלכות אסא עלה בעשא מלךישראל עליהודה ויבן אתהרמה לבלתי תת יוצא ובא לאסא מלך יהודה
Parsing verse: Parse this verse: לא תנסו אתיהוה אלהיכם כאשר נסיתם במסה
Parsing verse: Parse this verse: ולקח מדם הפר והזה באצבעו עלפני הכפרת קדמה ולפני הכפרת יזה שבעפעמים מןהדם באצבעו
Parsing verse: Parse this verse: כאלה תעשו ליום שבעת ימים לחם אשה ריחניחח ליהוה עלעולת התמיד יעשה ונסכו
Parsing verse: Parse this verse: וגםאמנה אחתי בתאבי הוא אך לא בתאמי ותהילי לאשה
Parsing verse: Parse this verse: חמת תנינם יינם וראש פתנים אכזר
Parsing verse: Parse this verse: ויאמר הנה נאאדני סורו נא אלבית עבדכם ולינ